In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
!pip install transformers
!pip install torch

In [6]:
import huggingface_hub

huggingface_hub.login(HF_TOKEN)

In [7]:
data = pd.read_csv('/content/drive/MyDrive/toxic_sentence_merged.csv')

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments

# RoBERTa 사전 학습 모델 로드 (2진 분류를 위한 출력층 추가)
model = AutoModelForSequenceClassification.from_pretrained("facebookai/roberta-base", num_labels=2)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("facebookai/roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at facebookai/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
from transformers import AutoTokenizer
import torch
# RoBERTa 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("facebookai/roberta-base")

# 텍스트를 토큰화
inputs = tokenizer(list(data['sentence']), padding=True, truncation=True, return_tensors="pt")
labels = torch.tensor(data['label'].values)  # 레이블을 텐서로 변환

In [12]:
from transformers import AutoModelForSequenceClassification

# RoBERTa 모델 로드 (2진 분류 작업을 위한 출력층 설정)
model = AutoModelForSequenceClassification.from_pretrained("facebookai/roberta-base", num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at facebookai/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
data = pd.read_csv('/content/drive/MyDrive/toxic_sentence_merged.csv')
data_temp = data.copy()

In [14]:
from torch.utils.data import Dataset
import torch

# Dataset 클래스를 상속하여 커스터마이징
class ContractDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # 텍스트를 토큰화
        encoding = self.tokenizer(self.texts[idx], padding='max_length', truncation=True, max_length=512, return_tensors='pt')

        # 토큰화된 텍스트와 레이블을 반환 (튜플 형식으로)
        item = {key: encoding[key].squeeze(0) for key in encoding}  # 차원 축소
        item['labels'] = torch.tensor(self.labels[idx])  # 레이블 추가
        return item

In [16]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import random_split

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',          # 결과 디렉토리
    num_train_epochs=10,              # 훈련 epoch 수
    per_device_train_batch_size=8,   # 배치 사이즈
    learning_rate=5e-5,
    eval_strategy="epoch",     # 평가 주기
    logging_dir='./logs',            # 로그 디렉토리
)

# Custom Dataset 객체 생성
train_dataset = ContractDataset(data_temp['sentence'].tolist(), data_temp['label'].tolist(), tokenizer)

train_size = int(0.7 * len(train_dataset))  # 70% 훈련
val_size = int(0.2 * len(train_dataset))    # 20% 검증
test_size = len(train_dataset) - train_size - val_size  # 나머지 10% 테스트

# 데이터셋을 훈련, 검증, 테스트로 나누기
train_dataset, eval_dataset, test_dataset = random_split(train_dataset, [train_size, val_size, test_size])

# Trainer에 Dataset 객체 전달
trainer = Trainer(
    model=model,                         # 모델
    args=training_args,                  # 훈련 인자
    train_dataset=train_dataset,         # 훈련 데이터셋
    eval_dataset=eval_dataset,          # 평가 데이터셋 (선택적으로 설정)
)

# 훈련 시작
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.546022
2,No log,0.587324
3,No log,0.669999
4,No log,0.667827
5,No log,0.666386
6,No log,0.665150
7,No log,0.667198
8,0.659600,0.660210
9,0.659600,0.632368
10,0.659600,0.450097


TrainOutput(global_step=680, training_loss=0.646235466003418, metrics={'train_runtime': 602.4337, 'train_samples_per_second': 8.997, 'train_steps_per_second': 1.129, 'total_flos': 1426061920051200.0, 'train_loss': 0.646235466003418, 'epoch': 10.0})

In [11]:
# 모델 로드
# wandb key : e06e8b1c2f26095018e41310344bfdafc2e1af80

In [17]:
# 평가
results = trainer.evaluate()

print("Evaluation results: ", results)

Evaluation results:  {'eval_loss': 0.45009690523147583, 'eval_runtime': 4.1992, 'eval_samples_per_second': 36.912, 'eval_steps_per_second': 4.763, 'epoch': 10.0}


In [24]:
# 예측 수행 (테스트 데이터셋을 기준으로)
predictions, label_ids, metrics = trainer.predict(test_dataset)

# 예측 결과 확인 (로짓에서 확률로 변환)
predicted_labels = torch.argmax(torch.tensor(predictions), dim=1)

# 예측 결과를 numpy 배열로 변환
predicted_labels = predicted_labels.numpy()

In [27]:
print(len(predicted_labels))  # predicted_labels의 길이

78


In [29]:
print(len(train_dataset), len(eval_dataset), len(test_dataset))
print(len(train_dataset) + len(eval_dataset), len(train_dataset) + len(eval_dataset) + len(test_dataset))

542 155 78
697 775


In [33]:
# predicted_label 컬럼을 뒤에 추가
data_temp['predicted_label'] = None  # 초기 값으로 None 또는 다른 값을 할당

# 예측 결과를 test_dataset에 해당하는 부분에 추가
start_index = len(train_dataset) + len(eval_dataset)  # 697
end_index = start_index + len(test_dataset)  # 775
data_temp.iloc[start_index:end_index, data_temp.columns.get_loc('predicted_label')] = predicted_labels

In [35]:
import pandas as pd

# 1. 'label'과 'predicted_label'이 일치하는 행 필터링 (test_dataset에 해당하는 부분)
matched_data = data_temp.iloc[len(train_dataset) + len(eval_dataset):][data_temp['label'] == data_temp['predicted_label']]
unmatched_data = data_temp.iloc[len(train_dataset) + len(eval_dataset):][data_temp['label'] != data_temp['predicted_label']]

# 2. 일치하는 확률 계산
match_probability = len(matched_data) / len(test_dataset)
unmatch_probability = len(unmatched_data) / len(test_dataset)

# 일치 확률 출력
print(f"일치 확률: {match_probability * 100:.2f}%")
print(f"일치하지 않을 확률: {unmatch_probability * 100:.2f}%")

일치 확률: 53.85%
일치하지 않을 확률: 46.15%


<ipython-input-35-99ab63507bc2>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matched_data = data_temp.iloc[len(train_dataset) + len(eval_dataset):][data_temp['label'] == data_temp['predicted_label']]
<ipython-input-35-99ab63507bc2>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unmatched_data = data_temp.iloc[len(train_dataset) + len(eval_dataset):][data_temp['label'] != data_temp['predicted_label']]
